In [1]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

### Spark session

In [2]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("DescriptiveAnalysis")\
    .getOrCreate()

sc=spark.sparkContext

### Read the data 

In [86]:
rdd= sc.textFile('../../Dataset/Google-Playstore.csv')
header= rdd.first()
rdd= rdd.filter(lambda x: x!=header)
# TODO: add suitable delimiter
# rdd.take(5)

# Avg Installations/ Free-Paid Apps

In [4]:
total_apps= rdd.count()
free_apps= rdd.filter(lambda x: x.split(',')[8] == 'True').count()
paid_apps= rdd.filter(lambda x: x.split(',')[8] == 'False').count()

print(f'Number of Free apps: {free_apps}')
print(f'Number of Paid apps: {paid_apps}')
print(f'Total Number of apps: {total_apps}')

In [5]:
def avg_installs(x):
    installations = x.split(',')[7]
    if x.split(',')[8] == 'True': return ('Free', installations)
    else:                         return ('Paid', installations)

installs_rdd = rdd.filter(lambda x:x!='')\
                .map(avg_installs)\
                .reduceByKey(lambda x, y: x + y)\
                .map(lambda x: (x[0], x[1] / free_apps if x[0] == 'Free' else x[1] / paid_apps))

installs_rdd.take(1)

# No. of Installs for each category

In [58]:
# installs= x[7], category= x[2]

installs_categ= rdd.filter(lambda x:x!='')\
    .map(lambda x: (x.split(',')[2], x.split(',')[7]))\
    .mapValues(lambda x: int(x))\
        .reduceByKey(lambda x,y:(x+y))\
    # .sortBy(lambda x: x[1], ascending=False)

installs_categ.take(5)      

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 75.0 failed 1 times, most recent failure: Lost task 0.0 in stage 75.0 (TID 295) (Mariem executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 830, in main
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 820, in process
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\rdd.py", line 5405, in pipeline_func
    return func(split, prev_func

In [ ]:
# input_data \
# .map(lambda row: (row[0], (row[1],row[2]))) \
# .groupByKey() \
# .map(lambda row: (
#         row[0],
#         sum([tup[0] for tup in list(row[1])]),
#         sum([tup[1] for tup in list(row[1])])
#     )
# ).collect()

# Developed apps per developer

In [87]:
developer_apps= rdd.filter(lambda x:x!='')\
    .map(lambda x: (x.split(',')[13], 1))\
    # .reduceByKey(lambda x,y:(x+y))\
    # .sortBy(lambda x: x[1], ascending=False)

developer_apps.take(5)

[('Jean Confident Irénée NIYIZIBYOSE', 1),
 ('5.0 and up', 1),
 ('Cabin Crew', 1),
 ('Climate Smart Tech2', 1),
 ('Rafal Milek-Horodyski', 1)]